# Stock Project - Real-Time Data Pipeline


In [7]:
%%sql
WITH latest AS (
    SELECT *
    FROM bronze_stock_minutes
    ORDER BY datetime DESC
    LIMIT 5000
)
SELECT *
FROM latest
ORDER BY datetime ASC;



StatementMeta(, 054d7e24-af9c-40c2-af7d-17e49d9ee38a, 11, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>

#### Get stock data

In [3]:
import yfinance as yf 
import pandas as pd 
from pyspark.sql.functions import col
# List of tickers you want to download
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

# Download 1-minute interval data for all tickers for the last trading day
# yfinance returns a multi-index DataFrame when multiple tickers are used
df = yf.download(
    tickers=tickers,
    interval="1m",
    period="1d",
    group_by="ticker"  # Organize data by ticker symbol
)

# Show the first rows
df.head()


StatementMeta(, 054d7e24-af9c-40c2-af7d-17e49d9ee38a, 7, Finished, Available, Finished)

[*********************100%***********************]  5 of 5 completed


Ticker                           AAPL                                      \
Price                            Open        High         Low       Close   
Datetime                                                                    
2026-01-07 14:30:00+00:00  263.269989  263.679993  262.049988  262.154999   
2026-01-07 14:31:00+00:00  262.250000  263.200012  262.059998  263.084991   
2026-01-07 14:32:00+00:00  263.179993  263.220001  262.640015  262.954987   
2026-01-07 14:33:00+00:00  262.950012  262.980011  262.619995  262.654999   
2026-01-07 14:34:00+00:00  262.670013  262.864990  262.140015  262.420013   

Ticker                                    TSLA                          \
Price                       Volume        Open        High         Low   
Datetime                                                                 
2026-01-07 14:30:00+00:00  1295765  435.885010  435.920013  433.309998   
2026-01-07 14:31:00+00:00   243664  433.549988  433.650909  431.600006   
2026-01-07 14:32:00+00:00   156857  432.459991  433.399902  432.250000   
2026-01-07 14:33:00+00:00   165063  433.049988  434.100006  432.000000   
2026-01-07 14:34:00+00:00    89670  432.000092  432.949890  431.670105   

Ticker                                          ...       GOOGL              \
Price                           Close   Volume  ...        Open        High   
Datetime                                        ...                           
2026-01-07 14:30:00+00:00  433.510101  1735910  ...  314.359985  316.399994   
2026-01-07 14:31:00+00:00  432.459991   427256  ...  315.670013  316.480011   
2026-01-07 14:32:00+00:00  432.975006   200792  ...  315.993988  316.046509   
2026-01-07 14:33:00+00:00  432.010010   315008  ...  315.660004  316.799988   
2026-01-07 14:34:00+00:00  432.877686   243720  ...  316.559998  316.989990   

Ticker                                                           MSFT  \
Price                             Low       Close  Volume        Open   
Datetime                                                                
2026-01-07 14:30:00+00:00  314.190002  315.730011  908778  479.559998   
2026-01-07 14:31:00+00:00  314.929993  315.980011  199434  479.980011   
2026-01-07 14:32:00+00:00  314.855011  315.630005  119234  478.190002   
2026-01-07 14:33:00+00:00  315.660004  316.500000  135287  479.299988   
2026-01-07 14:34:00+00:00  316.000000  316.209991  106679  481.519989   

Ticker                                                                 
Price                            High         Low       Close  Volume  
Datetime                                                               
2026-01-07 14:30:00+00:00  480.500000  478.910004  479.964996  615161  
2026-01-07 14:31:00+00:00  480.100006  478.059998  478.059998   72002  
2026-01-07 14:32:00+00:00  479.480011  477.950012  479.347992   60233  
2026-01-07 14:33:00+00:00  481.640015  479.299988  481.520599   78451  
2026-01-07 14:34:00+00:00  482.769989  481.190002  482.769989  169464  

[5 rows x 25 columns]

#### Flatten the multi-index DataFrame returned by yfinance

In [4]:
# Flatten the multi-index DataFrame returned by yfinance
# This creates a clean table with columns: ticker, datetime, open, high, low, close, volume
rows = []

# Loop through each ticker and extract its data
for ticker in tickers:
    # Extract the sub-DataFrame for the ticker
    temp = df[ticker].copy()
    temp = temp.reset_index()
    
    # Add the ticker column
    temp["ticker"] = ticker
    
    # Append to the list
    rows.append(temp)

# Combine all tickers into one DataFrame
df_flat = pd.concat(rows, ignore_index=True)

# Show the first rows
df_flat.head()


StatementMeta(, 054d7e24-af9c-40c2-af7d-17e49d9ee38a, 8, Finished, Available, Finished)

Price,Datetime,Open,High,Low,Close,Volume,ticker
0,2026-01-07 14:30:00+00:00,263.269989,263.679993,262.049988,262.154999,1295765,AAPL
1,2026-01-07 14:31:00+00:00,262.250000,263.200012,262.059998,263.084991,243664,AAPL
2,2026-01-07 14:32:00+00:00,263.179993,263.220001,262.640015,262.954987,156857,AAPL
3,2026-01-07 14:33:00+00:00,262.950012,262.980011,262.619995,262.654999,165063,AAPL
4,2026-01-07 14:34:00+00:00,262.670013,262.864990,262.140015,262.420013,89670,AAPL


#### Write the Spark DataFrame to a Delta table in the connected Lakehouse

In [5]:
# Convert pandas DataFrame to Spark DataFrame
df_spark = spark.createDataFrame(df_flat)

from delta.tables import DeltaTable
from pyspark.sql.functions import col

# Ensure correct data types
df_spark = df_spark.withColumn("Datetime", col("Datetime").cast("timestamp"))
df_spark = df_spark.withColumn("ticker", col("ticker").cast("string"))

target_table = "bronze_stock_minutes"

# Check if the table already exists
table_exists = (
    spark.sql("SHOW TABLES")
    .filter("tableName = 'bronze_stock_minutes'")
    .count()
)

if table_exists == 0:
    # First run → create the table
    print("Creating new Delta table 'stock_prices'...")
    df_spark.write.format("delta").saveAsTable(target_table)

else:
    # Table exists → perform MERGE (upsert)
    print("Merging new data into existing Delta table...")

    delta_table = DeltaTable.forName(spark, target_table)

    (
        delta_table.alias("t")
        .merge(
            df_spark.alias("s"),
            "t.ticker = s.ticker AND t.Datetime = s.Datetime"
        )
        .whenNotMatchedInsertAll()
        .execute()
    )

print("MERGE completed successfully.")


StatementMeta(, 054d7e24-af9c-40c2-af7d-17e49d9ee38a, 9, Finished, Available, Finished)

Merging new data into existing Delta table...
MERGE completed successfully.
